install libraries

In [1]:
# from google.colab import drive
# drive.mount('Oppositional_thinking')

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


In [3]:
# test gpu:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
torch.cuda.device_count()

cuda


2

In [4]:
import os
import torch
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from torch.optim import Adam, RMSprop
from transformers  import  get_scheduler
from sklearn.model_selection import KFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import KFold
import wandb
from datetime import datetime
import os
import shutil
import random
import numpy as np
from itertools import product


2024-05-20 08:57:58.109903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 08:57:58.110006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 08:57:58.263311: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


utils

In [5]:
# utils.py
def set_seed(seed):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)

    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)

def remove_previous_model(folder):
    dirs = [x for x in os.listdir(folder) if os.path.isdir(folder+os.sep+x)]
    for x in dirs:
        shutil.rmtree(folder+os.sep+x, ignore_errors=False, onerror=None)


def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in product(*vals):
        yield dict(zip(keys, instance))

# mydataset.py
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, mode='train'):
        self.encodings = encodings
        if mode !="train":
            self.labels=  [0]*len(encodings)
        else: self.labels = labels


    def __getitem__(self, idx):
        # item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

# datareader.py
import pandas as pd
import json

BINARY_MAPPING_CRITICAL_POS = {'CONSPIRACY': 0, 'CRITICAL': 1}
BINARY_MAPPING_CONSPIRACY_POS = {'CRITICAL': 0, 'CONSPIRACY': 1}

CATEGORY_MAPPING_CRITICAL_POS_INVERSE = {0: 'CONSPIRACY', 1: 'CRITICAL'}
CATEGORY_MAPPING_CONSPIRACY_POS_INVERSE = {0: 'CRITICAL', 1: 'CONSPIRACY'}

TRAIN_DATASET_ES="/kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_es_train.json"
TRAIN_DATASET_EN="/kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_en_train.json"
#TEST_DATASET_EN ="./dataset_oppositional/test/dataset_oppositional_test_nolabels/dataset_en_official_test_nolabels.json"
#TEST_DATASET_ES ="./dataset_oppositional/test/dataset_oppositional_test_nolabels/dataset_en_official_test_nolabels.json"


class PAN24Reader:
    def __init__(self):
        pass
    def read_json_file(self, path):
        dataset=[]
        print(f'Loading official JSON {path} dataset')
        with open(path, 'r', encoding='utf-8') as file:
            dataset = json.load(file)
        return dataset

    def load_dataset_classification(self, path, string_labels=False, positive_class='conspiracy'):
        dataset = self.read_json_file(path)
        # convert to a format suitable for classification
        texts = pd.Series([doc['text'] for doc in dataset])
        if string_labels:
            classes = pd.Series([doc['category'] for doc in dataset])
        else:
            if positive_class == 'conspiracy':
                binmap = BINARY_MAPPING_CONSPIRACY_POS
            elif positive_class == 'critical':
                binmap = BINARY_MAPPING_CRITICAL_POS
            else:
                raise ValueError(f'Unknown positive class: {positive_class}')
            classes = [binmap[doc['category']] for doc in dataset]
            classes = pd.Series(classes)
        ids = pd.Series([doc['id'] for doc in dataset])
        data = pd.DataFrame({"text": texts, "id": ids, "label": classes})
        return data


myReader=PAN24Reader()
es_train_df = myReader.load_dataset_classification(TRAIN_DATASET_ES, string_labels=False, positive_class='conspiracy')
en_train_df = myReader.load_dataset_classification(TRAIN_DATASET_EN, string_labels=False, positive_class='conspiracy')



Loading official JSON /kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_es_train.json dataset
Loading official JSON /kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_en_train.json dataset


fine_tunning.py

In [9]:
def training(_wandb, _model, _train_data, _val_data, _learning_rate, _optimizer_name, _schedule, _epochs,
             _tokenizer, _batch_size=32, _padding="max_length", _max_length=512, _truncation=True,
             _patience=10, _measure= "accuracy", _out=None):
    train_encodings = _tokenizer(_train_data["text"].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")
    val_encodings = _tokenizer(_val_data["text"].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")

    train, val = MyDataset(train_encodings, _train_data["label"].tolist()), MyDataset(val_encodings, _val_data["label"].tolist())

    train_dataloader, val_dataloader  = torch.utils.data.DataLoader(train, batch_size=_batch_size, shuffle=True), torch.utils.data.DataLoader(val, batch_size=_batch_size)


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    _wandb.log({"divice": str(device)})
    if use_cuda:
        model = _model.to(device)
        if torch.cuda.device_count() > 1:
            print(f"Usando {torch.cuda.device_count()} GPUs")
            model = torch.nn.DataParallel(model)
    best_measure, best_model_name, patience = None, None, 0
    training_stats = []
    
    # train_eval = evaluate.load("accuracy")
    train_eval = evaluate.load(f"Yeshwant123/{_measure}")

    lr_scheduler, optimizer = None, None
    #Here we can specify different methods to optmize the paarameters, initially we can consider Adam and RmsProp

    _wandb.log({"info": "Creating the Optimizer and Schedule "})

    lr_scheduler, optimizer = None, None
    if _optimizer_name == "adam":
        optimizer = Adam(_model.parameters(), lr=_learning_rate)
    elif _optimizer_name == "rmsprop":
        optimizer = RMSprop(_model.parameters(), lr=_learning_rate)

    #Here we can define different learning rate schedules, to variate de learning rate in each training step. Initially we use
    # can use linear learning rate schedule
    num_training_steps = _epochs * len(_train_data)
    if _schedule=="linear":
        lr_scheduler = get_scheduler(_schedule,
                                     optimizer=optimizer,
                                     num_warmup_steps=0,
                                     num_training_steps=num_training_steps)
    elif _schedule == "cosine":
        lr_scheduler = get_scheduler(_schedule,
                                     optimizer=optimizer,
                                     num_warmup_steps=0,
                                     num_training_steps=num_training_steps)
    

    for epoch in range(_epochs):
        if patience >= _patience: break
        total_loss_train, total_acc_train = 0, 0
        total_train_step = 0
        
        _model.train()
        
        for batch in train_dataloader:
            total_train_step += 1
            # print("Epoch ", epoch, "Batch", i)
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            if loss.dim() > 0:
                loss = loss.mean()  # Ensure the loss is a scalar
            loss.backward()
            total_loss_train += loss.item()
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            train_eval.add_batch(predictions=predictions, references=batch["labels"])
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        total_acc_train = train_eval.compute()

        total_eval_steps = 0
        total_loss_val, total_acc_val = 0, 0
        eval_metric = evaluate.load(f"Yeshwant123/{_measure}")
        model.eval()
        for batch in val_dataloader:
            total_eval_steps += 1
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
                loss = outputs.loss
                if loss.dim() > 0:
                    loss = loss.mean()  # Ensure the loss is a scalar
                total_loss_val += loss.item()
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
            eval_metric.add_batch(predictions=predictions, references=batch["labels"])
        total_acc_val = eval_metric.compute()

        if best_measure is None or (best_measure < total_acc_val[_measure]):  # here you must set your save weights
            if best_measure == None: _wandb.log({"info": "It's the first time (epoch) ******************"})
            elif best_measure < total_acc_val[_measure]:
                _wandb.log({"info": "In this epoch an improvement was achieved. (epoch) ******************"})

            best_measure = total_acc_val[_measure]
            try:
                os.makedirs(_out + os.sep + 'models', exist_ok=True)
            except OSError as error:
                _wandb.log({"info": "Directory '%s' can not be created"})
            # remove the directories
            remove_previous_model(_out + os.sep + 'models')
            best_model_name = _out + os.sep + 'models/bestmodel_epoch_{}'.format(epoch +1)
            _wandb.log({"info": "The current best model is " + best_model_name + " "+ str(best_measure)})

            os.makedirs(best_model_name, exist_ok=True)
            if isinstance(model, torch.nn.DataParallel):
                model.module.save_pretrained(best_model_name)
            else:
                model.save_pretrained(best_model_name)
            patience = 0
        else:
            patience += 1
        training_stats.append(
            {
                'epoch': epoch + 1,
                'Training Loss': total_loss_train / total_train_step,
                'Valid. Loss': total_loss_val / total_eval_steps,
                f'Valid.{_measure}': total_acc_val[_measure],
                f'Training.{_measure}': total_acc_train[_measure]
            }
        )
        
        _wandb.log({
            'epoch': epoch + 1,
            'train_loss': total_loss_train / len(train_dataloader),
            f'train_{_measure}': total_acc_train[_measure],
            'val_loss': total_loss_val / len(val_dataloader),
            f'val_{_measure}': total_acc_val[_measure]
        })

    if best_model_name != None:
        if isinstance(model, torch.nn.DataParallel):
            model = model.module.from_pretrained(best_model_name)
        else:
            model = model.from_pretrained(best_model_name)
        _wandb.log({"info": "The final model used to predict the labels of the testing datasets is " + best_model_name})

    df_stats = pd.DataFrame(data=training_stats)
    df_stats = df_stats.set_index('epoch')
    df_stats.to_csv(_out + os.sep + "training_stats.csv")

    _wandb.log({"info": df_stats})
    myplot = sns.lineplot(data=df_stats, palette="tab10", linewidth=2.5)
    fig = myplot.get_figure()
    fig.savefig(_out + os.sep + 'loss-figue.png')
    plt.close()
    return model

############################################################################################################################################################################3
#VALIDATION ON THE TEST SET

def validate(_wandb, _model, _test_data, _tokenizer, _batch_size=32, _padding="max_length", _max_length=512, _truncation=True, _measure="accuracy", evaltype=True):
    test_encodings = _tokenizer(_test_data['text'].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")
    _mode = "train" if evaltype else "test"
    test = MyDataset(test_encodings, _test_data["label"].tolist(), mode=_mode)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=_batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = _model.to(device)
        if torch.cuda.device_count() > 1:
            print(f"Usando {torch.cuda.device_count()} GPUs")
            model = torch.nn.DataParallel(model)
    
    eval_metric, out, k = None, None, 0

    if evaltype==True:
        eval_metric = evaluate.load(f"Yeshwant123/{_measure}")

    model.eval()
    total_loss = 0

    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
                outputs = _model(**batch)
                loss = outputs.loss
                if loss.dim() > 0:
                    loss = loss.mean()  # Ensure the loss is a scalar
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
                if k == 0:
                    out = predictions
                else:
                    out = torch.cat((out, predictions), 0)
                k += 1
                total_loss += loss.item()
                if evaltype:
                    eval_metric.add_batch(predictions=predictions, references=batch["labels"])

    if evaltype==True:
        total_acc_test = eval_metric.compute()
        test_mesure = total_acc_test[_measure]
        avg_test_loss = total_loss / len(test_dataloader)        # Log the test accuracy and loss to wandb
        _wandb.log({
            f'test_{_measure}': test_mesure,
            'test_avg_loss': avg_test_loss
        })
    return out


In [7]:
hyperparams_covered = []
# hyperparams_covered = [
#     {
#         "optimizer_name": "adam", 
#         "learning": 0.5e-5, 
#         "schedule": "linear",
#         "patience": 5, 
#         "epochs": 5,
#         "measure": "mcc",
#         "batch_size": 32, 
#         "max_length": 128
#     },
#     {
#         "optimizer_name": "adam", 
#         "learning": 0.5e-5, 
#         "schedule": "linear",
#         "patience": 5, 
#         "epochs": 10,
#         "measure": "mcc",
#         "batch_size": 32, 
#         "max_length": 128
#     },
# #############################################
#     {
#         "optimizer_name": "adam", 
#         "learning": 0.5e-5, 
#         "schedule": "linear",
#         "patience": 5, 
#         "epochs": 5,
#         "measure": "mcc",
#         "batch_size": 64, 
#         "max_length": 128
#     },
#     {
#         "optimizer_name": "adam", 
#         "learning": 0.5e-5, 
#         "schedule": "linear",
#         "patience": 5, 
#         "epochs": 10,
#         "measure": "mcc",
#         "batch_size": 64, 
#         "max_length": 128
#     },
#     {
#         "optimizer_name": "adam", 
#         "learning": 0.5e-5, 
#         "schedule": "linear",
#         "patience": 10, 
#         "epochs": 5,
#         "measure": "mcc",
#         "batch_size": 64, 
#         "max_length": 128
#     },
#     {
#         "optimizer_name": "adam", 
#         "learning": 0.5e-5, 
#         "schedule": "linear",
#         "patience": 10, 
#         "epochs": 10,
#         "measure": "mcc",
#         "batch_size": 64, 
#         "max_length": 128
#     },
# ]

main.py

In [8]:
# Get current date and time
current_datetime = datetime.now()

# Format it to include hours, minutes, and seconds
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

settings = wandb.Settings(start_method='thread', console='auto', mode='online')


wandb.login()

SEED=1234
set_seed(SEED)

preconfig = {
    0: {
        "lang": "english",
        "model_name": "roberta-base",
    },
    1: {
        "lang": "english",
        "model_name": "microsoft/deberta-base",
    },
    # 2: {
    #     "lang": "spanish",
    #     "model_name": "dccuchile/bert-base-spanish-wwm-uncased",
    # },
    # 3: {
    #     "lang": "spanish",
    #     "model_name": "PlanTL-GOB-ES/roberta-base-bne",
    # },
    # 4: {
    #     "lang": "spanish",
    #     "model_name": "bert-base-multilingual-uncased"
    # }
}

hyperparams = {
    "optimizer_name": ["adam", "rmsprop"], # ["adam", "rmsprop", "sgd"]
    "learning": [0.5e-5, 1e-6], # [0.5e-5, 1e-5, 0.5e-6, 1e-6
    "schedule": ["linear", "cosine"], # ["linear", "cosine", "constant"]
    "patience": [5, 10], # [3, 5, 10]
    "epochs": [5, 10], # [5, 10, 20]
    "measure": ["mcc"],
    "batch_size": [64], # [16, 32, 64, 128]
    "max_length": [128]
}

# Define KFold cross-validation
kf = KFold(n_splits=5)

# For each preconfiguration
for i, preconfig in preconfig.items():
    lang = preconfig["lang"]
    model_name = preconfig["model_name"]
    
    if lang == "spanish":
        X= es_train_df
    elif lang == "english":
        X= en_train_df
    
    print("Loading Tokenizer " + model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Initialize a counter for the runs
    run_counter = 0

    # For each hyperparameter configuration
    for config in product_dict(**hyperparams):
        run_counter += 1
        if config in hyperparams_covered:
            continue
        print(f"Iniciando proceso con los hiperparametros:\n{config}")
            
        # For each fold
        for fold, (train_index, val_index) in enumerate(kf.split(X)):
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]

            # Start a child run for this fold
            with wandb.init(project=f'lnr_oppositional_thinking_{formatted_datetime}',
                            entity='davidandreuroqueta',
                            group=f'{lang}_{model_name}',
                            job_type=f'hyperparam-tuning-{run_counter}',
                            name=f'{lang}_{model_name}_{run_counter}_fold_{fold}',
                            settings=settings) as fold_run:
                fold_run.config.update(preconfig)
                fold_run.config.update(config)
                fold_run.config.update({"SEED":SEED})

                # Log the fold number
                fold_run.config.update({"fold": fold + 1})
                print(f'Fold: {fold+1}')

                wandb.log({"info": "Loading Transformer Model " + model_name})
                model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
                

                # Train and validate your model, log metrics, etc.
                # ...
                # FINE-TUNING the model and obtaining the best model across all epochs
                fineTmodel = training(_wandb=fold_run, _model=model, _train_data=X_train, _val_data=X_val,
                                    _learning_rate=config["learning"], _optimizer_name=config["optimizer_name"],
                                    _schedule=config["schedule"], _epochs=config["epochs"], _tokenizer=tokenizer,
                                    _batch_size=config["batch_size"], _padding="max_length", _max_length=config["max_length"],
                                    _truncation=True, _patience=config["patience"], _measure=config["measure"], _out="./out")

                # VALIDATING OR PREDICTING on the test partition, this time I'm using the validation set, but you have to use the test set.
                preds = validate(_wandb=fold_run, _model=fineTmodel, _test_data=X_val, _tokenizer=tokenizer,
                                _batch_size=config["batch_size"], _padding="max_length", _max_length=config["max_length"],
                                _truncation=True, _measure=config["measure"], evaltype=True)

    # End the parent run
    # parent_run.finish()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Loading Tokenizer roberta-base


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Usando 2 GPUs


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Traceback (most recent call last):
  File "/tmp/ipykernel_34/434432311.py", line 101, in <module>
    fineTmodel = training(_wandb=fold_run, _model=model, _train_data=X_train, _val_data=X_val,
  File "/tmp/ipykernel_34/1890458922.py", line 65, in training
    lr_scheduler.step()
AttributeError: 'NoneType' object has no attribute 'step'


divice,cuda
info,Creating the Optimiz...


AttributeError: 'NoneType' object has no attribute 'step'

In [ ]:
# wandb.finish()